In [6]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
# SINCE THE MNIST DATASET IS IN 28X28, PADDING OF 2 IS PUT ON C1 LAYER
class LeNet5_Dict_Out(nn.Module):
    def __init__(self):
        super(LeNet5_Dict_Out,self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
        self.c1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1,padding=2)
        self.c2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1,padding=0)
        self.c3 = nn.Conv2d(in_channels=16,out_channels=120,kernel_size=5,stride=1,padding=0)
        # self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=2,stride=2)
        # self.fc0 = nn.Linear(in_features=400,out_features=120)
        self.fc1 = nn.Linear(in_features=120,out_features=84)
        self.fc2 = nn.Linear(in_features=84,out_features=10)

    def forward(self, img):
        out_dict = {}
        x = self.quant(img)
        # in 1x32x32 out 6x28x28
        x = self.c1(x)
        out_dict['c1'] = x
        # in 6x28x28 out 6x14x14
        # replace tanh with relu
        x = self.max_pool(x)
        out_dict['pool_c1'] = x
        x = self.relu(x)
        out_dict['relu_c1'] = x
        # in 6x14x14 out 16x10x10
        x = self.c2(x)
        out_dict['c2'] = x
        # in 16x10x10 out 16x5x5
        # replace tanh with relu
        x = self.relu(self.max_pool(x))
        out_dict['pool_c2'] = x
        # # in 16x5x5 out 400x1x1
        # x = torch.flatten(x,1)
        # x = self.tanh(self.fc0(x))
        # in 16x5x5 out 120x1x1
        # replace tanh with relu
        x = self.relu(self.c3(x))
        x = torch.flatten(x,1)
        out_dict['c3'] = x.int_repr()
        # replace tanh with relu
        x= self.fc1(x)
        out_dict['fc1'] = x
        x = self.relu(x)
        out_dict['fc1_relu'] = x
        x = self.fc2(x)
        out_dict['fc2'] = x
        x = self.dequant(x)
        return x, out_dict
    


In [7]:
from pathlib import Path
MODEL_NAME = "quantized_lenet5_mnist_20250703_1003.pth"
MODEL_PATH = Path("weights")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME


In [8]:
import torchvision
from torch.utils.data import DataLoader
# device-agnostic setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load test dataset
BATCH_SIZE = 32
test_dataset  = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transforms.ToTensor())
imgs = torch.stack([img for img, _ in test_dataset], dim=0)
mean = imgs.view(1, -1).mean(dim=1)    # or imgs.mean()
std = imgs.view(1, -1).std(dim=1)     # or imgs.std()
print(f"MNIST mean: {mean.item():.6f}")
print(f"MNIST std:  {std.item():.6f}")
# create Transformation (converting from Image class to Tensor and normalize)
mnist_transforms = transforms.Compose([transforms.ToTensor(),])
                                    #    transforms.Normalize(mean=mean, std=std)])

test_dataset  = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=mnist_transforms)
class_names = test_dataset.classes

test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)


# Step 1: Re-create model and set QAT config
model_fp32 = LeNet5_Dict_Out()
model_fp32.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(model_fp32 , inplace=True)

# Step 2: Convert to quantized version
net = torch.quantization.convert(model_fp32 .eval(), inplace=False)
# Step 3: Load quantized weights
net.load_state_dict(torch.load(MODEL_SAVE_PATH))


test_loss, test_acc = 0, 0

net.to(device)

net.eval()

MNIST mean: 0.132515
MNIST std:  0.310480


c:\Users\user\miniconda3\envs\ml\lib\site-packages\torch\ao\quantization\observer.py:244: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
c:\Users\user\miniconda3\envs\ml\lib\site-packages\torch\ao\quantization\utils.py:408: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(
c:\Users\user\miniconda3\envs\ml\lib\site-packages\torch\_utils.py:425: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


LeNet5_Dict_Out(
  (quant): Quantize(scale=tensor([0.0256]), zero_point=tensor([17]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (c1): QuantizedConv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), scale=0.13094060122966766, zero_point=68, padding=(2, 2))
  (c2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=0.33193832635879517, zero_point=83)
  (c3): QuantizedConv2d(16, 120, kernel_size=(5, 5), stride=(1, 1), scale=0.5925276279449463, zero_point=74)
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): QuantizedLinear(in_features=120, out_features=84, scale=0.5568221807479858, zero_point=67, qscheme=torch.per_tensor_affine)
  (fc2): QuantizedLinear(in_features=84, out_features=10, scale=1.0586904287338257, zero_point=84, qscheme=torch.per_tensor_affine)
)

In [20]:
import numpy as np

def to_c_array(arr,                # torch.Tensor or np.ndarray
               c_type="float",     # uint8_t, int8_t, int32_t, float…
               var_name="weights", # name in generated code
               max_per_line=16):   # pretty-print width
    """
    Returns a string like
        c_type weights[dim0][dim1]...[dimN] = { ... };
    ready to paste into C/C++.
    """
    # 1 · guaranteed ndarray
    if hasattr(arr, "cpu"):       # PyTorch tensor
        arr = arr.detach().cpu().numpy()
    elif not isinstance(arr, np.ndarray):
        arr = np.asarray(arr)

    # 2 · build dimension suffix
    dims = "".join(f"[{d}]" for d in arr.shape)
    header = f"{c_type} {var_name}{dims} = "

    # 3 · format recursively
    def rec(x, indent=""):
        if x.ndim == 1:                                   # leaf
            elems = ", ".join(map(str, x.tolist()))
            # soft line-wrap for readability
            if len(x) > max_per_line:
                # chunk into lines of <= max_per_line items
                chunks = [
                    ", ".join(map(str, x[i:i+max_per_line].tolist()))
                    for i in range(0, len(x), max_per_line)
                ]
                elems = (",\n" + indent + " ").join(chunks)
            return "{" + elems + "}"
        inner = ",\n".join(rec(y, indent + " ") for y in x)
        return "{\n" + indent + " " + inner + "\n" + indent + "}"

    body = rec(arr) + ";"
    return header + body

uint8_t image01[1][2][3][4] = {{{{0, 1, 2, 3},
 {4, 5, 6, 7},
 {8, 9, 10, 11}},
 {{12, 13, 14, 15},
 {16, 17, 18, 19},
 {20, 21, 22, 23}}}};


In [10]:
from matplotlib import pyplot as plt
from PIL import Image
IMAGE_NAME = "img_01.png"
IMAGE_PATH = Path(Path.cwd() / "results" / "images" / IMAGE_NAME)

weights = torch.load(MODEL_SAVE_PATH)
img = Image.open(IMAGE_PATH).convert('L')  # 'L' for grayscale
# plt.imshow(img, cmap='gray')
img_tensor = mnist_transforms(img)
# print(img_tensor*255)
torch.set_printoptions(profile="full")
print(net.quant(img_tensor).int_repr())
img_tensor.unsqueeze_(0)  # shape: [1, 1, 28, 28]
output, out_dict = net(img_tensor)
# print(output)
# print(out_dict['c1'])

tensor([[[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
          17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17],
         [17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,

In [24]:
c1_real_scale = (weights['quant.scale'] * weights['c1.weight'].q_scale()) / weights['c1.scale']
print(c1_real_scale.item())
print(weights['c1.scale'])
print(weights['c2.weight'].q_scale())
print(weights['c2.scale'])
c2_real_scale = (weights['c1.scale'] * weights['c2.weight'].q_scale()) / weights['c2.scale']
print(c2_real_scale.item())
c3_real_scale = (weights['c2.scale'] * weights['c3.weight'].q_scale()) / weights['c3.scale']
print('c3_real_scale',c3_real_scale.item())
print(weights['c3.zero_point'])
# print(weights['c2.weight'].q_zero_point())

0.0007882966310717165
tensor(0.1309)
0.005365171004086733
tensor(0.3319)
0.002116413554176688
c3_real_scale 0.0038776251021772623
tensor(74)


In [26]:
c1_bias_int32 = weights['c1.bias'] / (weights['quant.scale'].item()  * weights['c1.weight'].q_scale()) # convert bias to int32
print('c1_b',weights['c1.bias'])
c1_bias_int32 = torch.round(c1_bias_int32).to(torch.int32)
print('c1_bias_int32',c1_bias_int32)
c3_bias_int32 = weights['c3.bias'] / (weights['c2.scale'].item() * weights['c3.weight'].q_scale())
c3_bias_int32 = torch.round(c3_bias_int32).to(torch.int32)

print('c3_bias_int32',c3_bias_int32)

c1_b Parameter containing:
tensor([-0.1901, -0.1153,  0.0569, -0.3014, -0.3366, -0.0769],
       requires_grad=True)
c1_bias_int32 tensor([-1842, -1117,   551, -2920, -3261,  -745], dtype=torch.int32)
c3_bias_int32 tensor([ 35, -15,  14,  -7, -38,  32, -27,   9, -32, -36, -31,   8,   1,   7,
         -4, -59,  33,  -2,   4,  -6, -71, -20,  24,  -6, -26, -13,  -9,   2,
        -25, -16,  -1,   0, -33,  -1, -68,  -8, -23, -26, -27,  50,  -3,  16,
          4,   3,  15,  31, -30, -14, -37,   8,  -5, -16,  13, -48, -45, -18,
        -25, -13, -15,  63, -12, -12, -32, -37,  -8,  -5,  39, -11,   3, -37,
         -8,  11, -21, -33, -19, -15,  49, -48,  -3, -32,  24, -12, -17, -26,
         -9,   3,  -9, -13,  -7, -23, -15,  10,  20,   0, -45, -22,  -3,  30,
         11, -18, -17, -38, -23,  22,  41,  40,  -1,   6, -30,  -7, -19,  57,
        -20, -24, -53, -29,  10,  -8,  15, -26], dtype=torch.int32)


In [22]:
c3_weight = weights['c3.weight'].int_repr()
print(to_c_array(c3_weight,c_type="int8_t",var_name="c3_weight"))



int8_t c3_weight[120][16][5][5] = {
 {
  {
   {12, -8, -4, -7, 20},
{17, 2, -3, -10, 20},
{-3, -34, 22, -7, -2},
{-4, -5, 25, 22, -21},
{-2, 11, 46, 61, 9}
  },
{
   {35, 22, 10, -16, -7},
{-7, 4, -2, 6, -5},
{-23, -26, -6, 10, -6},
{11, -19, -33, -8, 16},
{-54, -68, -24, -27, 15}
  },
{
   {-11, -20, -13, -10, -9},
{-25, -16, -6, -1, 29},
{-7, 24, 20, -29, -11},
{-43, -18, -11, -41, -7},
{-13, -15, 22, 3, 4}
  },
{
   {-21, -6, -31, 13, 16},
{-18, -31, -30, -12, 6},
{-11, -11, 46, -5, -27},
{3, 8, 3, 0, -16},
{-12, 4, -35, 16, -33}
  },
{
   {-10, 12, 5, 17, 17},
{13, -15, -20, -4, 9},
{-20, -30, -35, 6, 13},
{13, 14, -10, 0, -30},
{1, 11, -3, -11, 1}
  },
{
   {14, 6, 12, 5, -3},
{16, -16, -15, -3, -19},
{-22, -38, -2, -40, -41},
{36, 16, -31, -80, -27},
{14, -10, -34, -13, 10}
  },
{
   {21, -2, 8, 11, 15},
{-6, -11, -8, -2, 14},
{13, -6, -35, -27, 7},
{12, -19, -8, 31, 8},
{-29, -20, -26, -2, -18}
  },
{
   {-15, -37, -38, -8, 7},
{1, 5, 22, -6, 1},
{30, 51, 13, -8, -15},
{-4, 13, 

## CHECK FOR C1 DIFFERENCE

In [30]:
c1_pytorch = out_dict['c1']
print(c1_pytorch.int_repr().to(torch.int8) - weights['c1.zero_point'])

# c1_manual = torch.load('img_01.png_c1.pt')
# torch.set_printoptions(profile="full")
# print(c1_pytorch[0,0,0:10,0:10].int_repr())
# print(torch.sum(c1_manual - c1_pytorch.int_repr())) 

tensor([[[[ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
          [ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
            -1,  -1,  -1,  -1,  -1,  -2,  -3,  -3,  -2,  -2,   0,   1,   0,  -1],
          [ -1,  -1,  

## CHECK FOR POOLING of c1 DIFFERENCE

In [68]:
c1_pool = out_dict['pool_c1']
print(c1_pool[0,0,:,:].int_repr())
c1_pool_manual = torch.load('img_01.png_pool_c1.pt')
print(c1_pool_manual[0,0,:,:])
print(torch.sum(c1_pool_manual - c1_pool.int_repr()))

tensor([[67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 67],
        [67, 67, 67, 68, 67, 72, 72, 72, 72, 71, 67, 67, 67, 67],
        [67, 67, 73, 73, 77, 77, 74, 67, 67, 67, 67, 67, 67, 67],
        [67, 71, 73, 72, 62, 66, 69, 69, 69, 70, 67, 67, 67, 67],
        [67, 67, 62, 68, 70, 71, 70, 67, 67, 68, 66, 67, 67, 67],
        [67, 67, 68, 71, 68, 67, 67, 67, 68, 66, 66, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 67, 67, 65, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 66, 68, 66, 68, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 68, 69, 65, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 69, 67, 67, 68, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 67, 69, 63, 68, 67, 67, 67, 67],
        [67, 67, 67, 67, 67, 67, 69, 67, 66, 68, 67, 67, 67, 67]],
       dtype=torch.uint8)
tensor([[67, 67, 67, 67, 67, 67, 67, 67, 67, 67, 

In [69]:
relu_c1 = out_dict['relu_c1']
relu_c1_manual = torch.load('img_01.png_relu_c1.pt')
print(torch.sum(relu_c1_manual-relu_c1.int_repr()))

tensor(34876)


In [31]:
import pandas as pd
conv_c2 = out_dict['c2']
print(conv_c2.shape)
print(conv_c2[0,0,:,:].int_repr())
for i in [7,15]:
    per_ch = torch.tensor(pd.read_csv(f'c2_{i}_c.csv', header=None).values)[:,:10]
    print(f'channel_{i}', torch.sum(per_ch.unsqueeze(0).unsqueeze(0) - conv_c2[0,i,:,:].int_repr()))
# conv_c2_manual = torch.load('img_01.png_c2.pt')
# print(torch.sum(conv_c2.int_repr() - conv_c2_manual))

torch.Size([1, 16, 10, 10])
tensor([[83, 83, 83, 82, 82, 80, 78, 77, 78, 79],
        [83, 83, 82, 82, 80, 79, 79, 78, 77, 78],
        [83, 82, 81, 81, 83, 85, 81, 79, 82, 84],
        [82, 80, 80, 80, 82, 79, 78, 80, 81, 81],
        [83, 83, 84, 81, 78, 80, 80, 76, 75, 78],
        [80, 79, 76, 72, 77, 81, 78, 71, 74, 80],
        [81, 78, 74, 76, 79, 81, 79, 76, 81, 85],
        [83, 80, 78, 78, 80, 81, 80, 81, 85, 84],
        [82, 82, 82, 82, 82, 81, 81, 83, 84, 83],
        [79, 79, 79, 78, 77, 78, 80, 82, 82, 82]], dtype=torch.uint8)
channel_7 tensor(0., dtype=torch.float64)
channel_15 tensor(0., dtype=torch.float64)


In [46]:
c2_pool_relu = out_dict['pool_c2']
for i in [2,3]:
    per_ch = torch.tensor(pd.read_csv(f'c2pr_{i}_c.csv', header=None).values)[:,:5]
    print(f'channel_{i}', torch.sum(per_ch.unsqueeze(0).unsqueeze(0) - c2_pool_relu[0,i,:,:].int_repr()))
# print(c2_pool_relu.int_repr())
# c2_pool_relu_manual = torch.load('img_01.png_relu_c2.pt')
# print(torch.sum(c2_pool_relu_manual - c2_pool_relu.int_repr()))

channel_2 tensor(0., dtype=torch.float64)
channel_3 tensor(0., dtype=torch.float64)


In [43]:
c3_conv_relu = out_dict['c3']
c3_manual = torch.tensor(pd.read_csv(f'c3_c.csv', header=None).values)[:,:120]
# print(c3_manual)
print(c3_manual - c3_conv_relu)
print(weights['c3.zero_point'])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
       dtype=torch.float64)
tensor(74)


In [72]:
fc1_weights = weights['fc1._packed_params._packed_params'][0].int_repr()
# print(fc1_weights)
# print(to_c_array(fc1_weights))
fc1_bias_int32 = weights['fc1._packed_params._packed_params'][1] / (weights['c3.scale'].item() * weights['fc1._packed_params._packed_params'][0].q_scale())
fc1_bias_int32 = torch.round(fc1_bias_int32).to(torch.int32)
print(fc1_bias_int32)
# c3_bias_int32 = weights['c3.bias'] / (weights['c2.scale'].item() * weights['c3.weight'].q_scale())
# c3_bias_int32 = torch.round(c3_bias_int32).to(torch.int32)
fc1_real_scale = (weights['c3.scale'] * weights['fc1._packed_params._packed_params'][0].q_scale()) / weights['fc1.scale']
print('fc1_real_scale',fc1_real_scale.item())
print(weights['fc1._packed_params._packed_params'][0].q_zero_point())

tensor([-23, -46,  61,  -9, -32, -11,  -4, -15,  19,  50,   6, -22,  17, -59,
        -33,  89, -43,  23,   0,  -5, -40,  52,  -8, -20, -56,  38, 105, -14,
         30,   9,  66,  21,  52,  14,  46,  -4, -21,  -9,  71, -29, -29, -41,
        -46, -28, -10,  75, -53,  32,  77,  -8,  49,  57, -28,  24,  12,   3,
        -34,  19, -46, -10, -41,  87, 127, -64, -35,  -8,  20,  26, -70,  12,
        -40,  76,   6, -88, -27,  16, -88,  19, -44, -26,  76,  66, -46,  17],
       dtype=torch.int32)
fc1_real_scale 0.004424193408340216
0


In [96]:
fc2_weights = weights['fc2._packed_params._packed_params'][0].int_repr()
# print(fc2_weights)
# print(to_c_array(fc2_weights))
fc2_bias_int32 = weights['fc2._packed_params._packed_params'][1] / (weights['fc1.scale'].item() * weights['fc2._packed_params._packed_params'][0].q_scale())
fc2_bias_int32 = torch.round(fc2_bias_int32).to(torch.int32)
print(fc2_bias_int32)
fc2_real_scale = (weights['fc1.scale'] * weights['fc2._packed_params._packed_params'][0].q_scale()) / weights['fc2.scale']
print(fc2_real_scale.item())
print(weights['fc2.zero_point'])

tensor([  3, -16, -33,   3, -12,   3,  -3,  -9,  42,   7], dtype=torch.int32)
0.0037336202803999186
tensor(84)


In [87]:
fc1_result = out_dict['fc1']
fc1_manual = torch.tensor(pd.read_csv(f'fc1_c.csv', header=None).values)[:,:84]
print(fc1_result.int_repr().to(torch.int) - fc1_manual)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)


In [88]:
fc1r_result = out_dict['fc1_relu']
fc1r_manual = torch.tensor(pd.read_csv(f'fc1r_c.csv', header=None).values)[:,:84]
print(fc1r_result.int_repr().to(torch.int) - fc1r_manual)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=torch.float64)


In [74]:

# from torchvision.utils import save_image
# torch.manual_seed(42)  # setting random seed

# # Create output folders
# output_img_dir = Path("results/images")
# output_img_dir.mkdir(parents=True, exist_ok=True)
# output_txt_path = Path("results/results.txt")

# # Clear or create the results.txt file
# with open(output_txt_path, 'w') as f:
#     f.write("idx, ground_truth, predicted\n")

# rows, cols = 2, 6
# fig = plt.figure(figsize=(12, 4))
# for i in range(1, (rows * cols) + 1):
#     random_idx = torch.randint(0, len(test_dataset), size=[1]).item()
#     img, label_gt = test_dataset[random_idx]
#     print(random_idx, img.shape, label_gt)
#     # print(img)
#     img_tensor = img.unsqueeze(dim=0).to(device)
#     # torch.save(img_tensor, f'img_{i:02d}.pt')  # Save the tensor for later use
#     # get the stuff with the highest confidence(?)
#     # label_pred = torch.argmax(net(img_tensor)).item()
#     with torch.no_grad():
#         print(net.quant(img_tensor).int_repr())

#         output, out_dict = net(img_tensor)
#         prob = torch.softmax(output, dim=1)[0]  # shape: [10]
#         label_pred = torch.argmax(prob).item()
#     # === Save image to PNG ===
#     img_filename = f"img_{i:02d}.png"
#     save_image(img, output_img_dir / img_filename)

#     # === Write result to text file ===
#     with open(output_txt_path, 'a') as f:
#         # f.write(f"{img_filename}, {class_names[label_gt]}, {class_names[label_pred]}\n")
#         prob_str = ", ".join([f"{class_names[i]}: {prob[i]:.4f}" for i in range(10)])
#         f.write(f"{img_filename}, GT: {class_names[label_gt]}, Pred: {class_names[label_pred]} → [{prob_str}]\n")    # === Plot image ===
#     fig.add_subplot(rows, cols, i)
#     img_display = img.permute(1, 2, 0)  # CxHxW → HxWxC
#     plt.imshow(img_display, cmap='gray')
#     color = 'g' if label_pred == label_gt else 'r'
#     plt.title(class_names[label_pred], color=color)
#     plt.axis('off')
#     break
# # print(out_dict)
# # torch.save(out_dict['c1'], f'img_01.png_c1_pytorch.pt')

# plt.tight_layout()
# plt.show()

In [97]:
fc2_result = out_dict['fc2']
print(fc2_result.int_repr())

tensor([[79, 77, 74, 82, 77, 91, 83, 74, 81, 79]], dtype=torch.uint8)
